In [1]:
import requests
import json
import pandas as pd
from datetime import datetime

import os
from dotenv import load_dotenv

In [10]:
# situational variables. To go https://goadmin.ifrc.org/docs/ to find API docs
api_url = "https://goadmin.ifrc.org/api/v2/personnel/"

rc_file = "ECOS Assignments Record.csv"

input_file_path = "file_inputs"
output_file_path = "file_outputs"

rc_file_and_path = os.path.join(input_file_path, rc_file)

executing_script = "compare_ifrc_to_local_dataset"



'2023-05-21_0706'

In [ ]:

for parts in [ ('one', 'two', 'three'),
               ('/', 'one', 'two', 'three'),
               ('/one', '/two', '/three'),
               ]:
    print parts, ':', os.path.join(*parts)

In [15]:
load_dotenv()
Authorization = os.environ.get("Authorization")

In [4]:
def get_everything_from_ifrcgo_endpoint(api_url, Authorization=None, offset=0, limit=50):
    
    
    if api_url.find("?") >= 0:
        api_url = api_url.split("?")[0]
        print(f"we're trying to keep this clean and simple, therefore your args are being ignored in favor of defaults.")
    
    def format_api_call(api_call, offset=0, limit=50):
        return f'{api_call}?limit={limit}&offset={offset}' if not "?" in api_call else api_call
    
   
    api_call = format_api_call(api_url)


    result_list = []

    while True:
        print(f'calling {api_call}')
        try:
            response_json = requests.get(api_call, headers={"Authorization":Authorization}).json()
            for r in response_json['results']:
                result_list.append(r)
        except Exception as e:
            print(f"{api_call} threw an exeption {e}")

        if response_json['next']:
            api_call = response_json['next']  

        else:
            return result_list
        
result_list = get_everything_from_ifrcgo_endpoint(api_url, Authorization=Authorization)
result_list_json_format = json.dumps(result_list)
print(f"you've got {len(result_list)} results in a list. Enjoy!")

calling https://goadmin.ifrc.org/api/v2/personnel/?limit=50&offset=0
calling https://goadmin.ifrc.org/api/v2/personnel/?limit=50&offset=50
calling https://goadmin.ifrc.org/api/v2/personnel/?limit=50&offset=100
calling https://goadmin.ifrc.org/api/v2/personnel/?limit=50&offset=150
calling https://goadmin.ifrc.org/api/v2/personnel/?limit=50&offset=200
calling https://goadmin.ifrc.org/api/v2/personnel/?limit=50&offset=250
calling https://goadmin.ifrc.org/api/v2/personnel/?limit=50&offset=300
calling https://goadmin.ifrc.org/api/v2/personnel/?limit=50&offset=350
calling https://goadmin.ifrc.org/api/v2/personnel/?limit=50&offset=400
calling https://goadmin.ifrc.org/api/v2/personnel/?limit=50&offset=450
calling https://goadmin.ifrc.org/api/v2/personnel/?limit=50&offset=500
calling https://goadmin.ifrc.org/api/v2/personnel/?limit=50&offset=550
calling https://goadmin.ifrc.org/api/v2/personnel/?limit=50&offset=600
calling https://goadmin.ifrc.org/api/v2/personnel/?limit=50&offset=650
calling h

In [7]:
# convert to a DF and filter for USA only
df = pd.json_normalize(result_list)
df_usa = df[df['country_from.iso3'] == 'USA'].copy()
df_usa[['role','start_date','name','country_from.iso3','deployment.country_deployed_to.name','deployment.event_deployed_to.countries',
        'country_from', 'country_to','deployment.event_deployed_to']]
df_usa.sample(2)

,start_date,end_date,type,molnix_id,molnix_tags,is_active,id,name,role,country_from.iso,...,deployment.event_deployed_to.tab_three_title,deployment.event_deployed_to.emergency_response_contact_email,deployment.event_deployed_to.name,deployment.event_deployed_to.slug,deployment.event_deployed_to.summary,deployment.id,deployment.comments,country_from,country_to,deployment.event_deployed_to
1156,2022-09-15T00:00:00Z,2022-12-15T00:00:00Z,rr,711.0,"[{'id': 1, 'molnix_id': 16, 'name': 'ASIAP', '...",True,1325,"Gaillis, Brigitte","Relief Coordinator, Monsoon Floods, Pakistan",US,...,None,None,Pakistan: Monsoon Floods,None,"<p><img src=""https://prddsgofilestorage.blob.c...",2873,None,NaN,NaN,NaN
896,2022-04-14T00:00:00Z,2022-07-07T00:00:00Z,rr,459.0,"[{'id': 24, 'molnix_id': 20, 'name': 'EURO', '...",True,1082,"Berg, Sally","CVA Roving, Ukraine Crises, Hungary.",US,...,None,None,Ukraine and impacted countries crisis,ukraine-crisis,"<iframe title=""Ukraine_Situation_Overview - Pu...",2853,None,NaN,NaN,NaN


In [8]:
def person_name_normalizer(name, fname_char_trunc=0):

    return_name = ''
    
    #first make sure name sequence is firstname lastname, removing middle names
    if name.find(',') > 0:
        nm = name.split(',')
        if len(nm) == 2:
            lname = nm[0].strip()
            fname = nm[1].strip()
            name = f"{fname} {lname}"
        else:
            print(f"we're probably dealing with middle names {name} - just returning first and last element - {nm[0].strip()} {nm[1].strip()}")
            name = f"{nm[-1].strip()} {nm[0].strip()}"

            
    #then remove middle name content (if it hasn't already been)
    nm = name.split(' ')
    if len(nm) == 2:
        lname = nm[1].strip()
        fname = nm[0].strip()

    else:
        print(f"we're probably dealing with middle names {name} - just returning first and last element")
        return_name = f"{nm[0].strip()} {nm[-1].strip()}"
        lname = nm[-1].strip()
        fname = nm[0].strip()
    
    # because of inconsistencies in how names can be recorded (Katherine vs Katy)
    # it may be desirable to just take the first n chars of the first name 
    fname = fname[:fname_char_trunc] if fname_char_trunc > 0 else fname
    return_name = f"{fname} {lname}"

    return return_name.lower()

## tests
assert person_name_normalizer("lastName, FirstName") == 'firstname lastname'
assert person_name_normalizer("FirstName lastName") == 'firstname lastname'
assert person_name_normalizer("lastName, FirstName MiddleName") == 'firstname lastname'
assert person_name_normalizer("FirstName MiddleName lastName") == 'firstname lastname'

assert person_name_normalizer("lastName, FirstName", 2) == 'fi lastname'
assert person_name_normalizer("FirstName lastName", 2) == 'fi lastname'
assert person_name_normalizer("lastName, FirstName MiddleName", 2) == 'fi lastname'
assert person_name_normalizer("FirstName MiddleName lastName", 2) == 'fi lastname'


    

df_usa['normalized_name'] = df_usa.name.apply(person_name_normalizer, fname_char_trunc=2)
df_usa['year'] = df_usa['start_date'].apply(lambda x: x[0:7])
df_usa['pseudo_key'] = df_usa['year'] + "_" + df_usa['normalized_name'].apply(lambda x: x.replace(" ","_"))
df_usa[df_usa['normalized_name'] == 'jo garro']

we're probably dealing with middle names FirstName MiddleName lastName - just returning first and last element
we're probably dealing with middle names FirstName MiddleName lastName - just returning first and last element
we're probably dealing with middle names FirstName MiddleName lastName - just returning first and last element
we're probably dealing with middle names FirstName MiddleName lastName - just returning first and last element
we're probably dealing with middle names David Andrew Strobel - just returning first and last element
we're probably dealing with middle names Katherine C. Wilkes - just returning first and last element
we're probably dealing with middle names Joe Prewitt Diaz - just returning first and last element
we're probably dealing with middle names William Carter Stone - just returning first and last element
we're probably dealing with middle names Virginia (Ginger) Clark - just returning first and last element
we're probably dealing with middle names William

,start_date,end_date,type,molnix_id,molnix_tags,is_active,id,name,role,country_from.iso,...,deployment.event_deployed_to.slug,deployment.event_deployed_to.summary,deployment.id,deployment.comments,country_from,country_to,deployment.event_deployed_to,normalized_name,year,pseudo_key
69,2019-02-25T00:00:00Z,2019-03-21T00:00:00Z,fact,NaN,[],True,203,Jonathan Garro,Information Management,US,...,None,,5,,NaN,NaN,NaN,jo garro,2019-02,2019-02_jo_garro
528,2020-11-11T18:33:05Z,2020-12-11T18:33:10Z,rr,NaN,[],True,608,Jonathan Garro,SIMS Coordinator,US,...,None,"<img src=""https://i.postimg.cc/NjxL49H4/Infogr...",84,None,NaN,NaN,NaN,jo garro,2020-11,2020-11_jo_garro
890,2022-03-04T00:00:00Z,2022-04-14T00:00:00Z,rr,303.0,"[{'id': 4, 'molnix_id': 223, 'name': 'IM', 'de...",True,931,"Garro, Jonathan","Humanitarian Information Analysis Coordinator,...",US,...,ukraine-crisis,"<iframe title=""Ukraine_Situation_Overview - Pu...",2853,None,NaN,NaN,NaN,jo garro,2022-03,2022-03_jo_garro
1308,2023-05-01T00:00:00Z,2023-05-26T00:00:00Z,rr,928.0,"[{'id': 4, 'molnix_id': 223, 'name': 'IM', 'de...",True,1534,"Garro, Jonathan","SIMS Coordinator, Sudan Internal Conflict, Sudan",US,...,None,"<p class=""MsoNormal""><span style=""color: #0000...",2920,None,NaN,NaN,NaN,jo garro,2023-05,2023-05_jo_garro


## Now load Red Cross File and generate DF

In [9]:
df_amrcross = pd.read_csv(rc_file_and_path)

df_amrcross['Start Date'] = pd.to_datetime(df_amrcross['Start Date'])
# this is dangerous... you should manually update blank start dates in the source file instead
# but I'm leaving it here so we can avoid stack traces...
df_amrcross['Start Date'] = df_amrcross['Start Date'].fillna(method="ffill") 
df_amrcross['normalized_name'] = (df_amrcross['Contact: First Name'] + ' ' + df_amrcross['Contact: Last Name']).apply(person_name_normalizer, fname_char_trunc=2)
df_amrcross['normalized_name'] = df_amrcross['normalized_name'].apply(lambda x : x.lower())
df_amrcross['pseudo_key'] = df_amrcross['Start Date'].apply(lambda x: x.strftime("%Y-%m")) + "_" + df_amrcross['normalized_name'].apply(lambda x: x.replace(" ","_"))
df_amrcross.sample(2)

we're probably dealing with middle names Christina (Tina) Eyre - just returning first and last element
we're probably dealing with middle names Christina (Tina) Eyre - just returning first and last element
we're probably dealing with middle names Christina (Tina) Eyre - just returning first and last element
we're probably dealing with middle names Christina (Tina) Eyre - just returning first and last element
we're probably dealing with middle names David (Drew) Strobel - just returning first and last element
we're probably dealing with middle names David (Drew) Strobel - just returning first and last element
we're probably dealing with middle names David (Drew) Strobel - just returning first and last element
we're probably dealing with middle names Raul Rivera Nuñez - just returning first and last element
we're probably dealing with middle names Raul Rivera Nuñez - just returning first and last element
we're probably dealing with middle names Raul Rivera Nuñez - just returning first an

,Contact: First Name,Contact: Last Name,Contact: Roster Status,Program/Project,Country,Assignment: Assignment Name,Mechanism,Start Date,End Date,Role Profile,normalized_name,pseudo_key
267,Stephen,Hagerich,On hold,NaN,NaN,Cash Transfer Program 2013-1-1,Bilateral,2013-01-01,12/31/2013,NaN,st hagerich,2013-01_st_hagerich
88,Daniel,Joseph,Active,Pacific Tropical Cyclone Pam-2015,Vanuatu,Information Management 2015-03-16,FACT,2015-03-18,4/18/2015,IM Coordinator,da joseph,2015-03_da_joseph


## standardize columns names for readability

In [10]:
for c in df_usa.columns:
    c_new = "ifrc_" + c
    c_new = c_new.lower().replace(" ","_")
    df_usa = df_usa.rename(columns={c:c_new})

In [11]:
for c in df_amrcross.columns:
    c_new = "amcross_" + c
    c_new = c_new.lower().replace(" ","_")
    df_amrcross = df_amrcross.rename(columns={c:c_new})

## outer join

In [ ]:
df_result = pd.merge(left=df_usa, right=df_amrcross, how='outer', left_on="ifrc_pseudo_key", right_on="amcross_pseudo_key", sort='start_date',\
                  suffixes=("_ifrc","_amercross")).copy()

## add a few helper columns to make it easier to pivot in excel

In [ ]:
def generate_unified_year(ifrc, rc):
    if ifrc == 0:
        return rc[0:4]
    else:
        return str(ifrc)[0:4]
    
df_result['unified_year'] = df_result.apply(lambda x: generate_unified_year(x.ifrc_pseudo_key, x.amcross_pseudo_key), axis=1)
df_result['one'] = 1
df_result[['ifrc_pseudo_key','amcross_pseudo_key']] = df_result[['ifrc_pseudo_key','amcross_pseudo_key']].fillna(0)

def determine_record_status(l,r):
    if (l == 0) and (r == 0):
        return "in neither"
    if (l != 0) and (r != 0):
        return "in both"
    if (l == 0):
        return "only in amcross"
    if (r == 0):
        return "only in ifrc"
    else:
        return "unexpected condition"
    
df_result['result'] = result.apply(lambda x: determine_record_status(x.ifrc_pseudo_key,x.amcross_pseudo_key), axis=1)
file = f"{executing_script}_{datetime.now().strftime('%Y-%m-%d_%H%M')}_res.csv"
output_file_name = os.path.join(input_file_path, file)
df_result.to_csv(output_file_name)